In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_csv(r"C:\Users\carlo\TrainingsTool\Tennis\Datasets\charting-m-points-from-2017-new.csv", encoding= 'unicode_escape', low_memory=False)

In [3]:
# only take matches where a specific player played
df = df.loc[df["match_id"].str.contains("Novak_Djokovic", case = True)]

In [4]:
# only keep rows where Djoko is serving
df = df[df.Serving == "ND"]

In [5]:
df = df.drop(["isAce", "Gm#", "match_id", "isUnret", "Svr", "Ret", "isRallyWinner", "isForced", "isUnforced", "isDouble", "Serving", "TBpt","Notes", "1stSV", "2ndSV", "1stIn", "2ndIn"], axis=1)

In [6]:
df = df[["Pt", "Set1", "Set2", "Gm1", "Gm2", "Pts", "TbSet", "TB?", "PtWinner", "isSvrWinner", "rallyCount", "1st", "2nd"]]

In [7]:
# seperate the Pts column to create pts_A and pts_B, splitting at the "-" seperator
df[["Pts1", "Pts2"]] = df.Pts.str.split("-", expand = True)

In [8]:
df = df.drop("Pts", axis=1)

In [9]:
df = df[["Pt", "Set1", "Set2", "Gm1", "Gm2", "Pts1", "Pts2", "TbSet", "TB?", "PtWinner", "isSvrWinner", "rallyCount", "1st", "2nd"]]

In [10]:
# Now we need to translate "AD" in Pts1 and Pts2 to "50"
columns = ["Pts1", "Pts2"]
for i in columns:
    df[str(i)] = df[str(i)].str.replace('AD', '50')

In [11]:
df = df.reset_index(drop=True)

In [12]:
test_df = df
test_df = test_df.astype({"1st": str, "2nd": str})

In [13]:
 # join the two ralley colums together into one column with "," seperator
test_df["1st&2nd"] = test_df[["1st", "2nd"]].apply(lambda x: "".join(x), axis=1)

In [14]:
test_df = test_df.drop(["1st", "2nd"], axis = 1)

In [15]:
ralley_df = test_df["1st&2nd"]

In [16]:
# creating a dataframe to store the shot_lists in
shot_list_dataframe = pd.DataFrame()

In [17]:
ralley_df_test = ralley_df.head(20)

In [18]:
possible_shots = ["f", "b", "r", "s", "v", "z", "o", "p", "y", "l", "m", "h", "i", "j", "k", "t", "u"]
serve_direction_second_serve = ["4", "5", "6"]
len_of_longest_ralley = 0
shot_list_dataframe = pd.DataFrame()

# i is a ralley
for i in ralley_df_test:
    # Initializing new shot for each new ralley
    shot = ""
    shot_list = []
    # every char in each ralley will be looked at
    for char in range(0, len(i)):
        # if the char is at position 3 in the ralley and its in serve direction list, its added to shot_list
        if i[char] in serve_direction_second_serve and char == 2:
            shot_list.append(shot)
            shot = ""
            shot = i[char]
             
        elif i[char] not in possible_shots and i[char] not in serve_direction_second_serve:
            shot = shot + str(i[char])
        # in all other cases, shot is done and appended to the shot_list
        else:
            shot_list.append(shot)
            shot = str(i[char])
    # the last shot has to be appended here, because it is not going into the loop again
    shot_list.append(shot)

    # if the last element of the shot list is "nan" we delete that because we dont want to predict "nan"
    if shot_list[-1][-1] == "n" and shot_list[-1][-2] == "a"  and shot_list[-1][-3] == "n" :
        shot_list[-1] = shot_list[-1].rstrip(shot_list[-1][-1])
        shot_list[-1] = shot_list[-1].rstrip(shot_list[-1][-1])
        shot_list[-1] = shot_list[-1].rstrip(shot_list[-1][-1])
        
    # finding the longest shot_list/ralley in the dataset
    if len(shot_list) > len_of_longest_ralley:
        len_of_longest_ralley = len(shot_list)

    shot_list_series = pd.Series(shot_list)
    shot_list_small_df = pd.DataFrame(shot_list)

    # now the shot list is beautiful and can add it to the empyt shot_list_dataframe
    shot_list_dataframe = pd.concat([shot_list_dataframe, shot_list_small_df], ignore_index=True, axis=1)

df = shot_list_dataframe

In [19]:
df = df.drop(0)
df = df.reset_index()
df = df.drop("index", axis = 1)

In [20]:
for i in range(0, len(ralley_df_test)):
    ralley = ralley_df_test[i]
    if ralley[-1] == "n" and ralley[-2] == "a" and ralley[-3] == "n":
        ralley_df_test[i] = ralley_df_test[i].rstrip(ralley_df_test[i][-1])
        ralley_df_test[i] = ralley_df_test[i].rstrip(ralley_df_test[i][-1])
        ralley_df_test[i] = ralley_df_test[i].rstrip(ralley_df_test[i][-1])

In [21]:
# df_each_char is created, each char is seperated into columns
shot_list_dataframe = pd.DataFrame()

# i is a ralley
for i in ralley_df_test:
    # Initializing new shot for each new ralley
    shot = ""
    shot_list = []
    # every char in each ralley will be looked at
    for char in range(0, len(i)):
        shot = i[char]
        shot_list.append(shot)
        shot = ""
        
    # the last shot has to be appended here, because it is not going into the loop again
    shot_list_series = pd.Series(shot_list)
    shot_list_small_df = pd.DataFrame(shot_list)

    # now the shot list is beautiful and can add it to the empyt shot_list_dataframe
    shot_list_dataframe = pd.concat([shot_list_dataframe, shot_list_small_df], ignore_index=True, axis=1)

df_each_char = shot_list_dataframe

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       8 non-null      object
 1   1       3 non-null      object
 2   2       1 non-null      object
 3   3       3 non-null      object
 4   4       14 non-null     object
 5   5       7 non-null      object
 6   6       26 non-null     object
 7   7       11 non-null     object
 8   8       2 non-null      object
 9   9       2 non-null      object
 10  10      1 non-null      object
 11  11      6 non-null      object
 12  12      2 non-null      object
 13  13      13 non-null     object
 14  14      1 non-null      object
 15  15      4 non-null      object
 16  16      2 non-null      object
 17  17      5 non-null      object
 18  18      7 non-null      object
 19  19      2 non-null      object
dtypes: object(20)
memory usage: 4.2+ KB
